In [1]:
import nltk.classify.util
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews # load data from nltk data set: movie reviews
from nltk.util import ngrams
from nltk.probability import FreqDist
from sklearn.naive_bayes import BernoulliNB
import heapq
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from random import shuffle
from nltk.classify import accuracy
from nltk.classify import SklearnClassifier
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('movie_reviews')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Drew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Drew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Drew\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Drew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
  return dict([(word, True) for word in words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
print('Positive review example: ' + ' '.join(movie_reviews.words(posids[0])))
print('Negative review example: ' + ' '.join(movie_reviews.words(negids[5])))

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()

Positive review example: films adapted from comic books have had plenty of success , whether they ' re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there ' s never really been a comic book like from hell before . for starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid ' 80s with a 12 - part series called the watchmen . to say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . the book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . in other words , don ' t dismiss this film because of its source . if you can get past the whole comic book thing , you might find another stumbling block in from hell ' s directors , albert and allen hughes . getting the hughes brothers t

train on 1500 instances, test on 500 instances
accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [3]:
##stop words


stop_words = set(stopwords.words('english'))

# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    return dict([(word, True) for word in filtered_words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


train on 1500 instances, test on 500 instances
accuracy: 0.712
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [4]:
##lemm
# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize words and convert sentences into feature vectors
def word_feats(words):
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
    return dict([(word, True) for word in lemmatized_words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors with lemmatization
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


train on 1500 instances, test on 500 instances
accuracy: 0.724
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
                  symbol = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0


In [5]:
##stemm
# Initialize the Porter stemmer
stemmer = PorterStemmer()

# Define a set of stopwords
stop_words = set(stopwords.words('english'))

# Function to preprocess words (remove stop words and stem)
def preprocess_word(word):
    if word.lower() not in stop_words:
        return stemmer.stem(word.lower())
    else:
        return None  # Return None for stop words

# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    return dict([(preprocess_word(word), True) for word in words if preprocess_word(word) is not None])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


train on 1500 instances, test on 500 instances
accuracy: 0.7
Most Informative Features
                outstand = True              pos : neg    =     13.6 : 1.0
                  ludicr = True              neg : pos    =     13.0 : 1.0
                uninvolv = True              neg : pos    =     12.3 : 1.0
                    plod = True              neg : pos    =     11.0 : 1.0
                  themat = True              pos : neg    =     11.0 : 1.0
                magnific = True              pos : neg    =      9.0 : 1.0
                  annual = True              pos : neg    =      9.0 : 1.0
                   inept = True              neg : pos    =      9.0 : 1.0
                   mulan = True              pos : neg    =      9.0 : 1.0
                    numb = True              neg : pos    =      9.0 : 1.0


In [6]:
##remove least informative
# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    return dict([(word, True) for word in words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)

# Get the most informative features
most_informative_features = classifier.most_informative_features()

# Remove the 10 least informative features
least_informative_features = heapq.nsmallest(10, most_informative_features, key=lambda x: x[1])

# Update feature sets by excluding the least informative features
def update_feature_sets(feature_set, least_informative_features):
    updated_feature_set = []
    for feat, label in feature_set:
        updated_feat = {k:v for k,v in feat.items() if k not in least_informative_features}
        updated_feature_set.append((updated_feat, label))
    return updated_feature_set

trainfeats_updated = update_feature_sets(trainfeats, [feat[0] for feat in least_informative_features])
testfeats_updated = update_feature_sets(testfeats, [feat[0] for feat in least_informative_features])

# Retrain the classifier with the updated feature sets
classifier_updated = NaiveBayesClassifier.train(trainfeats_updated)

# Evaluate the classifier's accuracy
print('accuracy after removing least informative features:', nltk.classify.util.accuracy(classifier_updated, testfeats_updated))


train on 1500 instances, test on 500 instances
accuracy after removing least informative features: 0.726


In [7]:
##remove <3 words
# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words, n):
    # Calculate word frequencies
    word_freq = FreqDist(words)

    # Select the N most frequent terms
    most_common_terms = heapq.nlargest(n, word_freq, key=word_freq.get)

    # Filter out rare/low-frequency words (frequency of term = 1, 2, 3, etc.)
    frequent_terms = [word for word in most_common_terms if word_freq[word] > 3]  # Adjust the threshold as needed

    return dict([(word, True) for word in words if word in frequent_terms])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
n = 1000  # Adjust the value of N as needed
negfeats = [(word_feats(movie_reviews.words(fileids=[f]), n), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f]), n), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


train on 1500 instances, test on 500 instances
accuracy: 0.66
Most Informative Features
                 fiction = True              pos : neg    =     10.3 : 1.0
                    girl = True              neg : pos    =      8.3 : 1.0
                 science = True              pos : neg    =      8.3 : 1.0
                     bad = True              neg : pos    =      8.1 : 1.0
                     war = True              pos : neg    =      7.8 : 1.0
                     why = True              neg : pos    =      7.8 : 1.0
                 another = True              pos : neg    =      7.7 : 1.0
                 cameron = True              pos : neg    =      7.7 : 1.0
                     own = True              pos : neg    =      7.7 : 1.0
                       + = True              pos : neg    =      7.0 : 1.0


In [8]:
##50 percentile
# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words, top_n, middle_percentile):
    freq_dist = FreqDist(words)
    
    # Select the N most frequent terms
    most_common_terms = [word for word, _ in freq_dist.most_common(top_n)]
    
    # Calculate the threshold for the middle percentile
    middle_start = int((len(most_common_terms) - len(most_common_terms) * middle_percentile / 2) / 2)
    middle_end = int((len(most_common_terms) + len(most_common_terms) * middle_percentile / 2) / 2)
    
    # Remove the middle 50% of the frequent terms
    selected_terms = most_common_terms[:middle_start] + most_common_terms[middle_end:]
    
    return dict([(word, True) for word in words if word in selected_terms])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f]), top_n=2000, middle_percentile=0.5), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f]), top_n=2000, middle_percentile=0.5), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3 / 4)
poscutoff = int(len(posfeats) * 3 / 4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


train on 1500 instances, test on 500 instances
accuracy: 0.742
Most Informative Features
             outstanding = True              pos : neg    =     27.7 : 1.0
                  finest = True              pos : neg    =     16.3 : 1.0
                lifeless = True              neg : pos    =     13.7 : 1.0
                 idiotic = True              neg : pos    =     13.0 : 1.0
                  turkey = True              neg : pos    =     12.3 : 1.0
             magnificent = True              pos : neg    =     11.7 : 1.0
              presumably = True              neg : pos    =     11.7 : 1.0
            respectively = True              pos : neg    =     11.7 : 1.0
                  avoids = True              pos : neg    =     10.3 : 1.0
            breathtaking = True              pos : neg    =     10.2 : 1.0


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
##TF-IDF
# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Extract document texts and labels
negdocs = [' '.join(movie_reviews.words(fileids=[f])) for f in negids]
posdocs = [' '.join(movie_reviews.words(fileids=[f])) for f in posids]
docs = negdocs + posdocs
labels = ['neg'] * len(negdocs) + ['pos'] * len(posdocs)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(docs, labels, test_size=0.25, random_state=42)

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# Fit TF-IDF vectorizer and transform documents
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Get feature names from TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Convert TF-IDF matrix to feature dictionaries
trainfeats = [(dict(zip(feature_names, row.toarray()[0])), label) for row, label in zip(X_train_tfidf, y_train)]
testfeats = [(dict(zip(feature_names, row.toarray()[0])), label) for row, label in zip(X_test_tfidf, y_test)]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


accuracy: 0.516
Most Informative Features
                     bad = 0.0               pos : neg    =      1.5 : 1.0
                    film = 0.0               neg : pos    =      1.4 : 1.0
                    plot = 0.0               pos : neg    =      1.4 : 1.0
                   movie = 0.0               pos : neg    =      1.4 : 1.0
                    life = 0.0               neg : pos    =      1.3 : 1.0
                    best = 0.0               neg : pos    =      1.3 : 1.0
                   great = 0.0               neg : pos    =      1.2 : 1.0
                   world = 0.0               neg : pos    =      1.2 : 1.0
                  script = 0.0               pos : neg    =      1.2 : 1.0
             performance = 0.0               neg : pos    =      1.2 : 1.0


In [18]:
##UNIGRAMS
# Function to convert sentences into unigram feature vectors
def unigram_feats(words):
    return dict([(word, True) for word in words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors (unigrams only)
negfeats = [(unigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(unigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('Accuracy (Unigrams Only):', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


Accuracy (Unigrams Only): 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [19]:
##uni and bi grams
# Function to convert sentences into unigram and bigram feature vectors
def unigram_bigram_feats(words):
    unigrams = unigram_feats(words)
    bigrams = [' '.join(bg) for bg in list(ngrams(words, 2))]  # Extract bigrams
    return {**unigrams, **dict([(bg, True) for bg in bigrams])}

# Convert sentences into feature vectors (unigrams and bigrams)
negfeats = [(unigram_bigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(unigram_bigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('Accuracy (Unigrams and Bigrams):', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


Accuracy (Unigrams and Bigrams): 0.74
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
              matt damon = True              pos : neg    =     13.7 : 1.0
               not funny = True              neg : pos    =     13.7 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
           a wonderfully = True              pos : neg    =     13.0 : 1.0
              and boring = True              neg : pos    =     13.0 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
             is terrific = True              pos : neg    =     13.0 : 1.0
             perfect for = True              pos : neg    =     12.3 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0


In [20]:
##uni, bi, trigrams
# Function to convert sentences into unigram, bigram, and trigram feature vectors
def unigram_bigram_trigram_feats(words):
    unigrams = unigram_feats(words)
    bigrams = [' '.join(bg) for bg in list(ngrams(words, 2))]  # Extract bigrams
    trigrams = [' '.join(tg) for tg in list(ngrams(words, 3))]  # Extract trigrams
    return {**unigrams, **dict([(bg, True) for bg in bigrams]), **dict([(tg, True) for tg in trigrams])}

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors (unigrams, bigrams, and trigrams)
negfeats = [(unigram_bigram_trigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(unigram_bigram_trigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('Accuracy (Unigrams, Bigrams, and Trigrams):', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


Accuracy (Unigrams, Bigrams, and Trigrams): 0.782
Most Informative Features
            of the worst = True              neg : pos    =     15.4 : 1.0
             magnificent = True              pos : neg    =     15.0 : 1.0
              matt damon = True              pos : neg    =     13.7 : 1.0
               not funny = True              neg : pos    =     13.7 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
           a wonderfully = True              pos : neg    =     13.0 : 1.0
              and boring = True              neg : pos    =     13.0 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
             is terrific = True              pos : neg    =     13.0 : 1.0
             perfect for = True              pos : neg    =     12.3 : 1.0


In [9]:
##TRIGRAMS
# Function to convert sentences into trigram feature vectors
def trigram_feats(words):
    trigrams = [' '.join(tg) for tg in list(ngrams(words, 3))]  # Extract trigrams
    return dict([(tg, True) for tg in trigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors (trigrams only)
negfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('Accuracy (Trigrams Only):', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


Accuracy (Trigrams Only): 0.81
Most Informative Features
            of the worst = True              neg : pos    =     15.4 : 1.0
             to be funny = True              neg : pos    =     11.7 : 1.0
              well , and = True              pos : neg    =     11.7 : 1.0
             work with . = True              neg : pos    =     11.0 : 1.0
               . too bad = True              neg : pos    =     10.3 : 1.0
              . while it = True              pos : neg    =      9.7 : 1.0
           not enough to = True              neg : pos    =      9.7 : 1.0
     , but unfortunately = True              neg : pos    =      9.0 : 1.0
          but the script = True              neg : pos    =      9.0 : 1.0
             on the edge = True              pos : neg    =      9.0 : 1.0


In [22]:
##BIGRAMS
# Function to convert sentences into bigram feature vectors
def bigram_feats(words):
    bigrams = [' '.join(bg) for bg in list(ngrams(words, 2))]  # Extract bigrams
    return dict([(bg, True) for bg in bigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors (bigrams only)
negfeats = [(bigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(bigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('Accuracy (Bigrams Only):', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


Accuracy (Bigrams Only): 0.754
Most Informative Features
              matt damon = True              pos : neg    =     13.7 : 1.0
               not funny = True              neg : pos    =     13.7 : 1.0
           a wonderfully = True              pos : neg    =     13.0 : 1.0
              and boring = True              neg : pos    =     13.0 : 1.0
             is terrific = True              pos : neg    =     13.0 : 1.0
             perfect for = True              pos : neg    =     12.3 : 1.0
                a boring = True              neg : pos    =     11.7 : 1.0
           absolutely no = True              neg : pos    =     11.0 : 1.0
              boring and = True              neg : pos    =     11.0 : 1.0
             brilliant , = True              pos : neg    =     11.0 : 1.0


In [23]:
##TRIGRAM w/o STOP WORDS
# Function to remove punctuation from words
def remove_punctuation(words):
    return [word for word in words if word not in string.punctuation]

# Function to convert sentences into trigram feature vectors
def trigram_feats(words):
    # Remove punctuation from words
    words = remove_punctuation(words)
    trigrams = [' '.join(tg) for tg in list(ngrams(words, 3))]  # Extract trigrams
    return dict([(tg, True) for tg in trigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors (trigrams only)
negfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]

# Train and evaluate the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(trainfeats)
print('Accuracy (Trigrams Only - Without Punctuation):', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


Accuracy (Trigrams Only - Without Punctuation): 0.816
Most Informative Features
            of the worst = True              neg : pos    =     15.4 : 1.0
             to be funny = True              neg : pos    =     12.3 : 1.0
            that s about = True              neg : pos    =     10.3 : 1.0
           not enough to = True              neg : pos    =      9.7 : 1.0
          but the script = True              neg : pos    =      9.0 : 1.0
             on the edge = True              pos : neg    =      9.0 : 1.0
                as he is = True              pos : neg    =      8.6 : 1.0
            to work with = True              neg : pos    =      8.4 : 1.0
         and even though = True              pos : neg    =      8.3 : 1.0
              but with a = True              pos : neg    =      8.3 : 1.0


In [10]:
##SVM
from nltk.classify import SklearnClassifier
from nltk.classify.util import accuracy
from sklearn.svm import SVC

# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    return dict([(word, True) for word in words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Support Vector Machine classifier
svm_classifier = SklearnClassifier(SVC())
svm_classifier.train(trainfeats)
print('accuracy:', accuracy(svm_classifier, testfeats))


train on 1500 instances, test on 500 instances
accuracy: 0.864


In [25]:
#SVM Trigrams
# Function to convert sentences into feature vectors by considering trigrams as features.
def trigram_feats(words):
    trigrams = [' '.join(tg) for tg in list(ngrams(words, 3))]  # Extract trigrams
    return dict([(tg, True) for tg in trigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors (trigrams only)
negfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Support Vector Machine classifier
svm_classifier = SklearnClassifier(SVC())
svm_classifier.train(trainfeats)
print('accuracy:', accuracy(svm_classifier, testfeats))


train on 1500 instances, test on 500 instances
accuracy: 0.508


In [26]:
##SVM stop words
# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    return dict([(word, True) for word in filtered_words])
# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Support Vector Machine classifier
svm_classifier = SklearnClassifier(SVC())
svm_classifier.train(trainfeats)
print('accuracy:', accuracy(svm_classifier, testfeats))


train on 1500 instances, test on 500 instances
accuracy: 0.866


In [27]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    return dict([(word, True) for word in words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Logistic Regression classifier
logistic_classifier = SklearnClassifier(LogisticRegression())
logistic_classifier.train(trainfeats)
print('accuracy:', accuracy(logistic_classifier, testfeats))


train on 1500 instances, test on 500 instances
accuracy: 0.896


In [28]:
#Logistic Regression
# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    return dict([(word, True) for word in filtered_words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Logistic Regression classifier
logistic_classifier = SklearnClassifier(LogisticRegression())
logistic_classifier.train(trainfeats)
print('accuracy:', accuracy(logistic_classifier, testfeats))


train on 1500 instances, test on 500 instances
accuracy: 0.868


In [29]:
#Logistic Regression + bigram
from sklearn.linear_model import LogisticRegression

def trigram_feats(words):
    trigrams = [' '.join(tg) for tg in list(ngrams(words, 2))]  # Extract trigrams
    return dict([(tg, True) for tg in trigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Logistic Regression classifier
logistic_classifier = SklearnClassifier(LogisticRegression())
logistic_classifier.train(trainfeats)
print('accuracy:', accuracy(logistic_classifier, testfeats))


train on 1500 instances, test on 500 instances
accuracy: 0.84


In [11]:
##BernoulliNB
stop_words = set(stopwords.words('english'))

# Function to convert sentences into feature vectors by considering every word as a feature.
def word_feats(words):
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    return dict([(word, True) for word in filtered_words])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Bernoulli Naive Bayes classifier
bernoulli_classifier = SklearnClassifier(BernoulliNB())
bernoulli_classifier.train(trainfeats)
print('accuracy:', accuracy(bernoulli_classifier, testfeats))

train on 1500 instances, test on 500 instances
accuracy: 0.802


In [32]:
#BernoulliNB + bigrams
stop_words = set(stopwords.words('english'))

# Function to convert sentences into feature vectors by considering every word as a feature.
def bigram_feats(words):
    bigrams = [' '.join(tg) for tg in list(ngrams(words, 2))]  # Extract biigrams
    return dict([(tg, True) for tg in bigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(bigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(bigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Bernoulli Naive Bayes classifier
bernoulli_classifier = SklearnClassifier(BernoulliNB())
bernoulli_classifier.train(trainfeats)
print('accuracy:', accuracy(bernoulli_classifier, testfeats))

train on 1500 instances, test on 500 instances
accuracy: 0.742


In [33]:
#BernoulliNB + trigrams
stop_words = set(stopwords.words('english'))

def trigram_feats(words):
    trigrams = [' '.join(tg) for tg in list(ngrams(words, 3))]  # Extract trigrams
    return dict([(tg, True) for tg in trigrams])

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Convert sentences into feature vectors
negfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(trigram_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]

# Create the dataset and split it into training and testing
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))

# Train and evaluate the Bernoulli Naive Bayes classifier
bernoulli_classifier = SklearnClassifier(BernoulliNB())
bernoulli_classifier.train(trainfeats)
print('accuracy:', accuracy(bernoulli_classifier, testfeats))

train on 1500 instances, test on 500 instances
accuracy: 0.536


In [12]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


stop_words = set(stopwords.words('english'))

# Function to preprocess and convert documents into a list of strings
def preprocess_documents(fileids):
    documents = []
    for fileid in fileids:
        words = movie_reviews.words(fileids=[fileid])
        filtered_words = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]
        documents.append(' '.join(filtered_words))
    return documents

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Preprocess documents
neg_docs = preprocess_documents(negids)
pos_docs = preprocess_documents(posids)
docs = neg_docs + pos_docs
labels = ['neg'] * len(neg_docs) + ['pos'] * len(pos_docs)

# Split the dataset into training and testing sets
docs_train, docs_test, labels_train, labels_test = train_test_split(docs, labels, test_size=0.25, random_state=42)

# Convert sentences into feature vectors using CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(docs_train)
X_test = vectorizer.transform(docs_test)

# Train and evaluate the Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, labels_train)
predicted_labels = clf.predict(X_test)
print('accuracy:', accuracy_score(labels_test, predicted_labels))


accuracy: 0.8


In [35]:

# Function to preprocess and convert documents into a list of strings
def preprocess_documents(fileids):
    documents = []
    for fileid in fileids:
        words = movie_reviews.words(fileids=[fileid])
        filtered_words = [word.lower() for word in words if word.isalpha()]  # Stop words are not removed
        documents.append(' '.join(filtered_words))
    return documents

# Extract positive and negative sentences
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

# Preprocess documents
neg_docs = preprocess_documents(negids)
pos_docs = preprocess_documents(posids)
docs = neg_docs + pos_docs
labels = ['neg'] * len(neg_docs) + ['pos'] * len(pos_docs)

# Split the dataset into training and testing sets
docs_train, docs_test, labels_train, labels_test = train_test_split(docs, labels, test_size=0.25, random_state=42)

# Convert sentences into feature vectors using CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(docs_train)
X_test = vectorizer.transform(docs_test)

# Train and evaluate the Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, labels_train)
predicted_labels = clf.predict(X_test)
print('accuracy:', accuracy_score(labels_test, predicted_labels))

accuracy: 0.8
